In [3]:
import os
import json
import requests

from web3 import Web3, HTTPProvider

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from utils.eth_blocks_utils import get_block_for_timestamp, get_timestamp_for_block

from pandas import Timestamp

In [4]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
web3 = Web3(HTTPProvider(f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}"))

In [5]:
api = "https://api.thegraph.com/subgraphs/name/convex-community/volume-mainnet"

In [6]:
pool_addr = "0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7"
with open("./3pool.json", 'r') as f:
    abi = json.load(f)
tripool = web3.eth.contract(address=pool_addr, abi=abi)

In [8]:
end_block = web3.eth.get_block_number()
end_block_timestamp = get_timestamp_for_block(end_block-2)
end_block_timestamp

1652346032

USDT balances got borky from the 7th of May 2022. So let's start from the 5th.

In [9]:
depegs_start = int(Timestamp(2022, 5, 5).timestamp())
query_start_block = get_block_for_timestamp(depegs_start)
query_start_block

14713964

In [10]:
step = 3600
time_steps = list(range(depegs_start, end_block_timestamp, step))
time_boxes = list(zip(time_steps, time_steps[1:]))
len(time_boxes)

177

In [11]:
blocks_to_inspect = []
c = len(time_boxes)
for time_box in time_boxes:
    print(f"left: {c}")
    c -= 1
    query = f"""
    {{
      swapEvents(
        where: {{
          pool: "{pool_addr.lower()}"
          timestamp_gte: {time_box[0]}
          timestamp_lt: {time_box[1]}
        }}
      ) {{
        block
      }}
    }}
    """
    r = requests.post(api, json={'query': query})
    queried_data = [int(i['block']) for i in dict(r.json())['data']['swapEvents']]
    blocks_to_inspect.extend(queried_data)
    data = list(set(blocks_to_inspect))

blocks_to_inspect

left: 177
left: 176
left: 175
left: 174
left: 173
left: 172
left: 171
left: 170
left: 169
left: 168
left: 167
left: 166
left: 165
left: 164
left: 163
left: 162
left: 161
left: 160
left: 159
left: 158
left: 157
left: 156
left: 155
left: 154
left: 153
left: 152
left: 151
left: 150
left: 149
left: 148
left: 147
left: 146
left: 145
left: 144
left: 143
left: 142
left: 141
left: 140
left: 139
left: 138
left: 137
left: 136
left: 135
left: 134
left: 133
left: 132
left: 131
left: 130
left: 129
left: 128
left: 127
left: 126
left: 125
left: 124
left: 123
left: 122
left: 121
left: 120
left: 119
left: 118
left: 117
left: 116
left: 115
left: 114
left: 113
left: 112
left: 111
left: 110
left: 109
left: 108
left: 107
left: 106
left: 105
left: 104
left: 103
left: 102
left: 101
left: 100
left: 99
left: 98
left: 97
left: 96
left: 95
left: 94
left: 93
left: 92
left: 91
left: 90
left: 89
left: 88
left: 87
left: 86
left: 85
left: 84
left: 83
left: 82
left: 81
left: 80
left: 79
left: 78
left: 77
left: 76
left

[14714120,
 14714304,
 14714570,
 14714768,
 14714810,
 14714861,
 14714984,
 14715185,
 14715410,
 14715629,
 14715668,
 14715575,
 14715570,
 14715572,
 14715952,
 14716019,
 14716040,
 14716155,
 14716411,
 14716564,
 14716609,
 14716710,
 14716967,
 14717586,
 14717626,
 14717910,
 14717926,
 14717917,
 14717958,
 14718315,
 14718164,
 14718238,
 14718168,
 14718658,
 14718671,
 14718783,
 14718712,
 14719137,
 14719137,
 14719137,
 14719106,
 14719137,
 14719025,
 14719137,
 14719423,
 14719349,
 14719421,
 14719491,
 14720442,
 14720468,
 14720690,
 14720676,
 14720955,
 14720795,
 14721063,
 14721347,
 14721597,
 14721897,
 14722016,
 14722434,
 14722472,
 14722448,
 14722736,
 14722793,
 14723067,
 14723413,
 14723175,
 14723628,
 14723581,
 14723526,
 14723468,
 14723583,
 14723420,
 14723844,
 14723830,
 14724075,
 14724114,
 14724009,
 14724009,
 14724442,
 14724214,
 14724712,
 14724706,
 14724919,
 14724928,
 14725000,
 14725012,
 14725141,
 14725251,
 14725409,
 14725332,

Get blocks wherever transactions happened.

In [12]:
num_blocks = len(blocks_to_inspect)
num_blocks

1726

We want token reserves and spot prices at swap_blocks (but first remove duplicates!)

In [19]:
data = []
done_blocks = []

In [20]:
dec_18 = 1000000000000000000
dec_6 = 1000000
for block in blocks_to_inspect:

    if block in done_blocks:
        continue

    print(f"blocks left: {num_blocks - len(done_blocks)}")

    block_timestamp = pd.Timestamp.fromtimestamp(get_timestamp_for_block(block))

    # spot prices - outputs are in 3crv so 18 decimals are divided
    usdt_dai_spot_price = tripool.functions.get_dy(2, 0, dec_6).call(block_identifier=block) * 1e-18
    usdt_usdc_spot_price = tripool.functions.get_dy(2, 1, dec_6).call(block_identifier=block) * 1e-6

    # reserves
    dai_reserve = tripool.functions.balances(0).call(block_identifier=block) * 1e-18
    usdc_reserve = tripool.functions.balances(1).call(block_identifier=block) * 1e-6
    usdt_reserve = tripool.functions.balances(2).call(block_identifier=block) * 1e-6

    data.append(
        {
            "block": block,
            "timestamp": block_timestamp,
            "dai_usdt_spot_price": usdt_dai_spot_price,
            "usdc_usdt_spot_price": usdt_usdc_spot_price,
            "dai_reserve": dai_reserve,
            "usdc_reserve": usdc_reserve,
            "usdt_reserve": usdt_reserve,
        }
    )
    done_blocks.append(block)

blocks left: 1726
blocks left: 1725
blocks left: 1724
blocks left: 1723
blocks left: 1722
blocks left: 1721
blocks left: 1720
blocks left: 1719
blocks left: 1718
blocks left: 1717
blocks left: 1716
blocks left: 1715
blocks left: 1714
blocks left: 1713
blocks left: 1712
blocks left: 1711
blocks left: 1710
blocks left: 1709
blocks left: 1708
blocks left: 1707
blocks left: 1706
blocks left: 1705
blocks left: 1704
blocks left: 1703
blocks left: 1702
blocks left: 1701
blocks left: 1700
blocks left: 1699
blocks left: 1698
blocks left: 1697
blocks left: 1696
blocks left: 1695
blocks left: 1694
blocks left: 1693
blocks left: 1692
blocks left: 1691
blocks left: 1690
blocks left: 1689
blocks left: 1688
blocks left: 1687
blocks left: 1686
blocks left: 1685
blocks left: 1684
blocks left: 1683
blocks left: 1682
blocks left: 1681
blocks left: 1680
blocks left: 1679
blocks left: 1678
blocks left: 1677
blocks left: 1676


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
df_data = pd.DataFrame(data)
df_data

In [ ]:
df_data.to_csv("./usdt_pool_imbalance_reserves_spot_price.csv")